In [ ]:
import nibabel as nib
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from matplotlib import animation
from IPython.display import HTML
from scipy.stats import zscore

In [ ]:
# === Paths ===
subject_id = "100307"
base_path = os.path.expanduser(f"~/hcp_data/subjects/{subject_id}")
rfmri_path = os.path.join(base_path, "rfMRI")
t1w_path = os.path.join(base_path, "T1w", "T1w_acpc_dc_restore.nii.gz")
behavioral_csv = os.path.expanduser("~/hcp_data/metadata/behavioral.csv")

In [ ]:
# === Load behavioral data ===
behavior_df = pd.read_csv(behavioral_csv)
neo_cols = ["NEOFAC_O", "NEOFAC_C", "NEOFAC_E", "NEOFAC_A", "NEOFAC_N"]
personality = behavior_df[behavior_df["Subject"] == int(subject_id)][neo_cols]
print("\nNEO-FFI Personality Scores:")
print(personality.head())

In [ ]:
# === Load T1w structural MRI ===
t1w_img = nib.load(t1w_path)
t1w_data = t1w_img.get_fdata()
print("\nT1w shape:", t1w_data.shape)

In [ ]:
# === Show central anatomical slices ===
fig, axs = plt.subplots(1, 3, figsize=(12, 4))
axs[0].imshow(np.rot90(t1w_data[t1w_data.shape[0] // 2, :, :]), cmap="gray")
axs[0].set_title("Sagittal")
axs[1].imshow(np.rot90(t1w_data[:, t1w_data.shape[1] // 2, :]), cmap="gray")
axs[1].set_title("Coronal")
axs[2].imshow(np.rot90(t1w_data[:, :, t1w_data.shape[2] // 2]), cmap="gray")
axs[2].set_title("Axial")
for ax in axs:
    ax.axis("off")
plt.suptitle("T1w Structural MRI")
plt.show()

In [ ]:
# === Load one rs-fMRI scan ===
run_name = "REST1_LR"
rfmri_data = nib.load(os.path.join(rfmri_path, f"rfMRI_{run_name}.nii.gz")).dataobj[...,:600]
#rfmri_data = rfmri_img.get_fdata() not loading whole thing just first 100 time points
print(f"\nrs-fMRI shape ({run_name}):", rfmri_data.shape)

In [ ]:
# === Load T1w structural MRI ===
t1w_img = nib.load(t1w_path)
t1w_data = t1w_img.get_fdata()
print("\nT1w shape:", t1w_data.shape)

In [ ]:
#Pick a slice in Z
z_idx = 45

# Set up the figure
fig, ax = plt.subplots()
frame = np.rot90(rfmri_data[:, :, z_idx, 0])
im = ax.imshow(frame, cmap='gray', animated=True)
ax.axis('off')

# Animation update function
def update(frame_idx):
    frame = np.rot90(rfmri_data[:, :, z_idx, frame_idx])
    im.set_array(frame)
    ax.set_title(f"Time {frame_idx}")
    return [im]

# Create animation object
anim = animation.FuncAnimation(fig, update, rfmri_data.shape[3], interval=60, blit=False)

# Display in Jupyter
HTML(anim.to_jshtml())

In [ ]:
std_volume = np.std(rfmri_data, axis=3)

# Choose slice range to animate
z_start = 30
z_end = 60
slice_range = range(z_start, z_end)

# Set up figure
fig, ax = plt.subplots(figsize=(6, 6))
img = ax.imshow(np.rot90(std_volume[:, :, z_start]), cmap='hot', vmin=0, vmax=np.percentile(std_volume, 99))
ax.set_title(f"Temporal STD - Slice z={z_start}")
ax.axis("off")

# Define update function
def update(frame):
    z = slice_range[frame]
    img.set_array(np.rot90(std_volume[:, :, z]))
    ax.set_title(f"Temporal STD - Slice z={z}")
    return [img]

# Create animation
anim = animation.FuncAnimation(fig, update, frames=len(slice_range), interval=120, blit=False)

# Display animation inline
HTML(anim.to_jshtml())

In [ ]:
# === Compute basic functional connectivity matrix ===
# Step 1: average over all voxels within a 10x10x10 cube at center (for simplicity)
cube = rfmri_data[
    rfmri_data.shape[0] // 2 - 5 : rfmri_data.shape[0] // 2 + 5,
    rfmri_data.shape[1] // 2 - 5 : rfmri_data.shape[1] // 2 + 5,
    rfmri_data.shape[2] // 2 - 5 : rfmri_data.shape[2] // 2 + 5,
    :
]
voxels = cube.reshape(-1, cube.shape[-1])  # shape = (n_voxels, time)
voxels = zscore(voxels, axis=1)  # z-score each voxel time series
fc_matrix = np.corrcoef(voxels)  # correlation matrix

In [ ]:
# Step 2: Show FC matrix
plt.imshow(fc_matrix, cmap="coolwarm", vmin=-1, vmax=1)
plt.colorbar(label="Correlation")
plt.title("Functional Connectivity Matrix\n(100 voxels in center cube)")
plt.show()